In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# import needed external libraries
import os
import sys
import sqlite3 as sql
import pandas as pd
import csv

import json
import dateutil

# Include the `src` module in the module path for imports
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# import custom methods


### Import Data

In [214]:
df_comments = pd.read_csv('../../src/data/csv_exports/comments.csv')

df_trivia = pd.read_csv('../../src/data/csv_exports/trivia.csv')

df_metadata = pd.read_csv('../../src/data/csv_exports/metadata.csv')

df_brands = pd.read_csv('../../src/data/csv_exports/brands.csv')

In [195]:
print(df_comments.columns, '\n', df_trivia.columns, '\n', df_metadata.columns)

Index(['article_num', 'comment_id', 'commenter_name', 'comment_published',
       'comment_text', 'comment_likes', 'comment_flag', 'parent_id'],
      dtype='object') 
 Index(['Unnamed: 0', 'label', 'a', 'q'], dtype='object') 
 Index(['article_id', 'article_cat', 'article_title', 'article_author',
       'article_published', 'article_modified', 'comm_count', 'word_count'],
      dtype='object')


In [182]:
titles = df_metadata['article_title']

In [44]:
q_comments = []

for i in df_comments['comment_text']:
    if '?' in i:
        q_comments.append(i)
        
trivia_questions = df_trivia['q']

print('No of question-comments:', len(q_comments), ';  No of trivia questions:', len(trivia_questions))

No of question-comments: 25640 ;  No of trivia questions: 474


In [204]:
df_comments.loc[1]

article_num                                                       3605
comment_id                                                           4
commenter_name                                              jpsusantyo
comment_published                             2017-01-11T16:23:30.000Z
comment_text         Would love to see some sort of forum. Already ...
comment_likes                                                       10
comment_flag                                                         0
parent_id                                                            0
Name: 1, dtype: object

In [207]:
qc = pd.DataFrame(columns=df_comments.columns)

In [208]:
qc

,article_num,comment_id,commenter_name,comment_published,comment_text,comment_likes,comment_flag,parent_id


In [212]:
for i, c in enumerate(df_comments['comment_text']):
    if '?' in c:
        print(type(df_comments.loc[i]))
            

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.S

KeyboardInterrupt: 

In [210]:
qc

,article_num,comment_id,commenter_name,comment_published,comment_text,comment_likes,comment_flag,parent_id


In [200]:
pd.DataFrame(q_comments).to_csv('../../src/data/csv_exports/q_comments.csv')

In [19]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction import text

import nltk
import sklearn
import pandas as pd
import string, re
import urllib

In [115]:
def q_stemmer(q):
    
    pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
    stop_words = set(nltk.corpus.stopwords.words('english')).union({'watch'})
    stemmer = nltk.SnowballStemmer("english")

    q_tokens_raw = nltk.regexp_tokenize(q, pattern)
    q_tokens = [i.lower() for i in q_tokens_raw]
    q_tokens_stopped = [w for w in q_tokens if not w in stop_words]
    q_stem = [stemmer.stem(word) for word in q_tokens_stopped]
    
    q_stem = ' '.join(q_stem)
    
    return q_stem

In [217]:
def q_stemmer(q):
    
    pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
    stop_words = set(nltk.corpus.stopwords.words('english')).union({'watch'}).union(set(df_brands))
    stemmer = nltk.SnowballStemmer("english")

    q_tokens_raw = nltk.regexp_tokenize(q, pattern)
    q_tokens = [i.lower() for i in q_tokens_raw]
    q_tokens_stopped = [w for w in q_tokens if not w in stop_words]
    q_stem = q_tokens_stopped
#     q_stem = [stemmer.stem(word) for word in q_tokens_stopped]
    
    q_stem = ' '.join(q_stem)
    
    return q_stem

In [218]:
q_stemmer(q_comments[71])

'ceram durabl'

In [33]:
q_comments[0]

"FINALLY!  comments on the app.  I love you guys.  When are we going to be able to search other people's watch collections?"

In [116]:
q_stemmer(q_comments[71])

'ceram durabl'

In [219]:
qc_stem = []
qt_stem = []
titles_stem = []

for q in q_comments:
    qc_stem.append(q_stemmer(q))

for q in trivia_questions:
    qt_stem.append(q_stemmer(q))
    
for q in titles:
    titles_stem.append(q_stemmer(q))    

In [186]:
titles_stem

[]

In [220]:
counts = text.CountVectorizer()

In [221]:
qt_fitted = counts.fit_transform(qt_stem)

qc_fitted = counts.transform(qc_stem)
titles_fitted = counts.transform(titles_stem)

In [222]:
tfidf = sklearn.feature_extraction.text.TfidfVectorizer()

In [223]:
qt_tfidf = tfidf.fit_transform(qt_stem)
qc_tfidf = tfidf.transform(qc_stem)
titles_tfidf = tfidf.transform(titles_stem)

In [224]:
vocab = counts.vocabulary_
print(len(vocab), type(vocab))

1651 <class 'dict'>


In [225]:
vocab_tfidf = tfidf.vocabulary_
print(len(vocab_tfidf), type(vocab_tfidf))

1651 <class 'dict'>


In [226]:
sim_titles = pd.DataFrame(cosine_similarity(qc_tfidf, titles_tfidf))

In [227]:
sim_titles.describe()

,0,1,2,3,4,5,6,7,8,9,...,6491,6492,6493,6494,6495,6496,6497,6498,6499,6500
count,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,...,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.0,25640.000000,25640.000000,25640.000000,25640.000000
mean,0.025408,0.023907,0.010032,0.022114,0.004471,0.010055,0.009050,0.009270,0.024890,0.000851,...,0.021414,0.011164,0.014510,0.009118,0.009747,0.0,0.021002,0.000987,0.008014,0.001963
std,0.063983,0.059999,0.037315,0.056958,0.029873,0.032992,0.037833,0.031921,0.072114,0.014105,...,0.058240,0.036243,0.051952,0.050871,0.033648,0.0,0.067980,0.018590,0.046209,0.025709
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
max,0.603130,0.535064,0.532176,0.444683,0.520117,0.456938,0.677567,0.406296,0.801262,0.617834,...,0.708145,0.439083,0.690148,0.734042,0.447445,0.0,0.728604,1.000000,1.000000,0.918939


In [230]:
max_c = []
max_c_scores = []

for index, i in enumerate(sim_titles.columns):
    max_c.append(q_comments[sim_titles[i].idxmax()])
    max_c_scores.append(sim_titles[i].max())
    if max_c_scores[index] > 0.95:
        print(max_c[index], titles[index], max_c_scores[index])


Have you ever seen a sapphire watch in the metal? The Craziest Watch Strap We've Ever Seen 1.0000000000000002
Was that 6200 John Mayer's? With John Goldberger 1.0
Am I a troll for hating this Tourbillon iwatch? The Greubel Forsey Double Tourbillon Asymétrique Explained 1.0
What about McQueen's Explorer II? (lol) The Linde Werdelin SpidoLite II Tech 1.0
So you care for neither fish nor fowl, just Submariners? A (Tropical) Tudor Submariner Issued To The WWII Battleship USS Nereus, And Engraved As Such 1.0
Can it be machine washed and tumble dried? The MB&F Legacy Machine N°2 Explained 1.0
No blue or black? :( The Tudor Heritage Chrono Blue 1.0
What about Konstantin Chaikin’s Easter clock? Here are 331 Vintage Wall Clocks 1.0
Any guesses on what the blancpain will be? Fifty fathoms? The Blancpain Fifty Fathoms Bathyscaphe 1.0000000000000002
Wast it made for Putin? :-P Made In The USA: Horween Shell Cordovan Straps Now Available In The HODINKEE Shop 1.0
Seriously ? We are talking about wat

In [228]:
sim_tf = pd.DataFrame(cosine_similarity(qc_tfidf, qt_tfidf))

In [229]:
sim_tf.describe()

,0,1,2,3,4,5,6,7,8,9,...,464,465,466,467,468,469,470,471,472,473
count,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,...,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000
mean,0.004845,0.014123,0.004975,0.015377,0.022479,0.013489,0.006948,0.009785,0.004629,0.001247,...,0.010538,0.005208,0.018522,0.014762,0.004184,0.004315,0.005923,0.013625,0.008429,0.007294
std,0.033825,0.048112,0.020230,0.043647,0.041269,0.047590,0.029330,0.030057,0.019394,0.010196,...,0.026585,0.022267,0.039484,0.032102,0.021390,0.021324,0.024009,0.030308,0.034317,0.026400
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.035832,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.020699,0.016331,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.759663,0.595559,0.306611,0.500029,0.395457,0.714413,0.501900,0.353233,0.381109,0.346522,...,0.345521,0.394862,0.415827,0.367922,0.433120,0.371083,0.385059,0.383689,0.497938,0.444603


In [123]:
similarities = cosine_similarity(qc_fitted, qt_fitted)

In [124]:
sims = pd.DataFrame(similarities)

In [132]:
sims.describe()

,0,1,2,3,4,5,6,7,8,9,...,464,465,466,467,468,469,470,471,472,473
count,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,...,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000,25640.000000
mean,0.005687,0.018952,0.006886,0.023427,0.030990,0.017925,0.006988,0.013772,0.007995,0.003200,...,0.023661,0.005400,0.022618,0.026950,0.004373,0.005221,0.008882,0.018162,0.009876,0.012114
std,0.038482,0.059749,0.025209,0.066009,0.057051,0.056841,0.029147,0.039444,0.028940,0.021809,...,0.051878,0.023732,0.046782,0.051451,0.022008,0.023443,0.034840,0.038425,0.039690,0.041993
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.048507,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.026228,0.042108,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.707107,0.577350,0.258199,0.603023,0.424264,0.632456,0.436436,0.365148,0.333333,0.500000,...,0.471405,0.516398,0.377964,0.353553,0.471405,0.326599,0.408248,0.397360,0.447214,0.577350


In [157]:
len(sims.columns)

474

In [168]:
sims[0].max()

0.7071067811865475

In [169]:
max_c = []
max_c_scores = []

for index, i in enumerate(sims.columns):
    max_c.append(q_comments[sims[i].idxmax()])
    max_c_scores.append(sims[i].max())

In [175]:
max_c_scores.index(max(max_c_scores))

136

In [178]:
trivia_questions[136]

'What is a "form" watch?'

In [180]:
max_c[136]

'Various forms of STD?'

In [127]:
q_comments[13869]

'What do you call the deepsea?'

In [ ]:
for index, i in enumerate(max_c):
    print(trivia_questions[index])
    print(i)
    print('\n')

In [131]:
qc_stem[1215]

'call deepsea'

In [177]:
qt_stem[0]

'call moon'

In [144]:
from random import sample 


In [146]:
print(sample(q_comments,3)) 


["Terrible. Glad no one was harmed. Hopefully their inventory mechanism is such that they can trace what has gone missing by serial? Certainly if it's the artwork there are few pieces more uniquely traceable. Want to see these guys apprehended.", 'And what are the size of those screws compared to the ones in a standard watch movement?', 'Link?']


In [147]:
q_comments.index('Link?')

631

In [148]:
q_comments[631]

'Link?'

In [153]:
df_comments[df_comments['comment_text'] == 'Link?']

,article_num,comment_id,commenter_name,comment_published,comment_text,comment_likes,comment_flag,parent_id
3648,3732,4211,PaulAC,2017-02-11T19:45:55.000Z,Link?,0,0,4206
54742,4945,58784,QuartzCrisis81,2018-06-11T22:32:20.000Z,Link?,1,0,58772
73912,5323,78878,Desmo,2018-11-28T16:27:50.000Z,Link?,0,0,78872
